# 17. Doporučovací Systémy s Transformery

**Autor:** Praut s.r.o. - AI Integration & Business Automation

V tomto notebooku se naučíme vytvářet moderní doporučovací systémy pomocí Transformer modelů a embedding technik.

## Obsah
1. Úvod do doporučovacích systémů
2. Content-based doporučování s embeddingsy
3. Collaborative filtering s Transformery
4. Hybridní doporučovací systém
5. Produkční implementace

In [ ]:
# Instalace knihoven
!pip install transformers sentence-transformers torch pandas numpy scikit-learn faiss-cpu -q

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, field
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Kontrola GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Používám zařízení: {device}")

## 1. Úvod do doporučovacích systémů

Existují tři hlavní přístupy:

| Přístup | Popis | Výhody | Nevýhody |
|---------|-------|--------|----------|
| Content-based | Doporučuje podobné položky | Funguje bez historie | Cold-start problém |
| Collaborative | Využívá chování uživatelů | Objevuje nové vzory | Potřebuje hodně dat |
| Hybridní | Kombinuje oba přístupy | Nejlepší výsledky | Složitější implementace |

In [ ]:
# Vytvoření syntetických dat pro e-shop

def generate_ecommerce_data(n_users: int = 500, n_products: int = 200, n_interactions: int = 5000):
    """
    Generuje syntetická data pro e-shop doporučovací systém.
    """
    np.random.seed(42)
    
    # Kategorie produktů
    categories = ['Elektronika', 'Oblečení', 'Sport', 'Domácnost', 'Knihy', 'Kosmetika']
    
    # Produkty
    products = []
    product_names = [
        # Elektronika
        'Smartphone Samsung Galaxy', 'iPhone 15 Pro', 'Notebook Lenovo', 'Tablet iPad',
        'Sluchátka Sony', 'Bluetooth reproduktor', 'Chytré hodinky', 'Herní konzole',
        # Oblečení
        'Zimní bunda', 'Džíny Levis', 'Běžecké boty Nike', 'Sportovní triko',
        'Kožená peněženka', 'Sluneční brýle', 'Pánské hodinky', 'Dámská kabelka',
        # Sport
        'Fitness náramek', 'Yoga podložka', 'Činky set', 'Běžecký pás',
        'Cyklistická helma', 'Tenisová raketa', 'Fotbalový míč', 'Plavecké brýle',
        # Domácnost
        'Kávovar DeLonghi', 'Robotický vysavač', 'LED lampa', 'Sada nožů',
        'Mixér KitchenAid', 'Žehlička Philips', 'Topinkovač', 'Rychlovarná konvice',
        # Knihy
        'Python programování', 'Machine Learning základy', 'Byznys strategie',
        'Osobní rozvoj', 'Fantasy román', 'Detektivka', 'Kuchařka', 'Cestopis',
        # Kosmetika
        'Parfém Chanel', 'Pleťový krém', 'Šampon profesionální', 'Rtěnka MAC',
        'Holící strojek', 'Deodorant sprej', 'Tělové mléko', 'Makeup set'
    ]
    
    for i, name in enumerate(product_names):
        cat_idx = i // 8  # 8 produktů na kategorii
        products.append({
            'product_id': i,
            'name': name,
            'category': categories[cat_idx % len(categories)],
            'price': np.random.randint(100, 5000),
            'description': f"Kvalitní {name.lower()} pro náročné zákazníky. Kategorie: {categories[cat_idx % len(categories)]}."
        })
    
    # Doplnění na n_products
    while len(products) < n_products:
        idx = len(products)
        cat = np.random.choice(categories)
        products.append({
            'product_id': idx,
            'name': f"Produkt {idx}",
            'category': cat,
            'price': np.random.randint(50, 3000),
            'description': f"Produkt z kategorie {cat}."
        })
    
    products_df = pd.DataFrame(products[:n_products])
    
    # Uživatelé s preferencemi
    users = []
    for i in range(n_users):
        preferred_cats = np.random.choice(categories, size=np.random.randint(1, 4), replace=False)
        users.append({
            'user_id': i,
            'preferred_categories': list(preferred_cats),
            'avg_price_preference': np.random.choice(['low', 'medium', 'high'])
        })
    
    users_df = pd.DataFrame(users)
    
    # Interakce (zobrazení, přidání do košíku, nákup)
    interactions = []
    interaction_types = ['view', 'add_to_cart', 'purchase']
    interaction_weights = [0.6, 0.3, 0.1]  # Většina jsou zobrazení
    
    for _ in range(n_interactions):
        user_id = np.random.randint(0, n_users)
        user = users[user_id]
        
        # Uživatel má tendenci kupovat z preferovaných kategorií
        if np.random.random() < 0.7:  # 70% interakcí z preferovaných kategorií
            pref_products = products_df[products_df['category'].isin(user['preferred_categories'])]
            if len(pref_products) > 0:
                product_id = pref_products.sample(1)['product_id'].values[0]
            else:
                product_id = np.random.randint(0, n_products)
        else:
            product_id = np.random.randint(0, n_products)
        
        interaction_type = np.random.choice(interaction_types, p=interaction_weights)
        
        # Přiřazení skóre podle typu interakce
        score_map = {'view': 1, 'add_to_cart': 3, 'purchase': 5}
        
        interactions.append({
            'user_id': user_id,
            'product_id': product_id,
            'interaction_type': interaction_type,
            'score': score_map[interaction_type],
            'timestamp': pd.Timestamp.now() - pd.Timedelta(days=np.random.randint(0, 90))
        })
    
    interactions_df = pd.DataFrame(interactions)
    
    return products_df, users_df, interactions_df


# Generování dat
products_df, users_df, interactions_df = generate_ecommerce_data()

print(f"Produkty: {len(products_df)}")
print(f"Uživatelé: {len(users_df)}")
print(f"Interakce: {len(interactions_df)}")
print(f"\nPříklad produktů:")
print(products_df.head())
print(f"\nPříklad interakcí:")
print(interactions_df.head())

## 2. Content-based doporučování s embeddingsy

In [ ]:
class ContentBasedRecommender:
    """
    Content-based doporučovač využívající Sentence Transformers.
    Doporučuje produkty na základě podobnosti popisů.
    """
    
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        print(f"Načítání modelu {model_name}...")
        self.encoder = SentenceTransformer(model_name)
        self.product_embeddings = None
        self.product_ids = None
        self.index = None
        
    def fit(self, products_df: pd.DataFrame, text_column: str = 'description'):
        """
        Vytvoří embeddingy pro všechny produkty.
        """
        print(f"Vytváření embeddingů pro {len(products_df)} produktů...")
        
        # Kombinace názvu a popisu pro lepší reprezentaci
        texts = (products_df['name'] + ". " + products_df[text_column]).tolist()
        
        # Vytvoření embeddingů
        self.product_embeddings = self.encoder.encode(
            texts,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        self.product_ids = products_df['product_id'].values
        
        # Vytvoření FAISS indexu pro rychlé vyhledávání
        dimension = self.product_embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner product = cosine pro normalizované vektory
        
        # Normalizace embeddingů
        faiss.normalize_L2(self.product_embeddings)
        self.index.add(self.product_embeddings)
        
        print(f"Index vytvořen: {self.index.ntotal} produktů, {dimension}D embeddingy")
        
    def get_similar_products(
        self,
        product_id: int,
        n_recommendations: int = 5
    ) -> List[Tuple[int, float]]:
        """
        Najde podobné produkty k danému produktu.
        """
        # Najdi index produktu
        idx = np.where(self.product_ids == product_id)[0]
        if len(idx) == 0:
            return []
        
        # Získej embedding produktu
        query_embedding = self.product_embeddings[idx[0]:idx[0]+1]
        
        # Vyhledej podobné
        scores, indices = self.index.search(query_embedding, n_recommendations + 1)
        
        # Odstraň původní produkt a vrať výsledky
        results = []
        for score, i in zip(scores[0], indices[0]):
            if self.product_ids[i] != product_id:
                results.append((int(self.product_ids[i]), float(score)))
        
        return results[:n_recommendations]
    
    def search_products(
        self,
        query: str,
        n_results: int = 5
    ) -> List[Tuple[int, float]]:
        """
        Vyhledá produkty na základě textového dotazu.
        """
        # Zakóduj dotaz
        query_embedding = self.encoder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(query_embedding)
        
        # Vyhledej
        scores, indices = self.index.search(query_embedding, n_results)
        
        return [(int(self.product_ids[i]), float(score)) 
                for score, i in zip(scores[0], indices[0])]


# Vytvoření a trénování content-based doporučovače
content_recommender = ContentBasedRecommender()
content_recommender.fit(products_df)

# Test - podobné produkty
print("\n--- Test podobných produktů ---")
test_product_id = 0  # Smartphone Samsung Galaxy
test_product = products_df[products_df['product_id'] == test_product_id].iloc[0]
print(f"Produkt: {test_product['name']} ({test_product['category']})")
print(f"\nPodobné produkty:")

similar = content_recommender.get_similar_products(test_product_id, n_recommendations=5)
for pid, score in similar:
    product = products_df[products_df['product_id'] == pid].iloc[0]
    print(f"  {product['name']} ({product['category']}) - skóre: {score:.3f}")

# Test - textové vyhledávání
print("\n--- Test vyhledávání ---")
query = "sportovní vybavení pro fitness"
print(f"Dotaz: '{query}'")
print(f"\nVýsledky:")

search_results = content_recommender.search_products(query, n_results=5)
for pid, score in search_results:
    product = products_df[products_df['product_id'] == pid].iloc[0]
    print(f"  {product['name']} ({product['category']}) - skóre: {score:.3f}")

## 3. Collaborative filtering s Transformery

In [ ]:
class TransformerCollaborativeFilter(nn.Module):
    """
    Transformer-based collaborative filtering model.
    Modeluje sekvenci interakcí uživatele pro predikci dalších preferencí.
    """
    
    def __init__(
        self,
        n_users: int,
        n_items: int,
        embedding_dim: int = 64,
        n_heads: int = 4,
        n_layers: int = 2,
        dropout: float = 0.1,
        max_seq_length: int = 50
    ):
        super().__init__()
        
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        
        # Embeddingy
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items + 1, embedding_dim, padding_idx=0)  # +1 pro padding
        self.position_embedding = nn.Embedding(max_seq_length, embedding_dim)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=n_heads,
            dim_feedforward=embedding_dim * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Output layers
        self.output_layer = nn.Sequential(
            nn.Linear(embedding_dim * 2, embedding_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim, n_items)
        )
        
        self._init_weights()
        
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, std=0.02)
    
    def forward(
        self,
        user_ids: torch.Tensor,
        item_sequences: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        """
        Args:
            user_ids: (batch_size,)
            item_sequences: (batch_size, seq_length)
            attention_mask: (batch_size, seq_length) - True pro platné pozice
            
        Returns:
            scores: (batch_size, n_items) - skóre pro každou položku
        """
        batch_size, seq_length = item_sequences.shape
        
        # Embeddingy položek + pozice
        positions = torch.arange(seq_length, device=item_sequences.device).unsqueeze(0).expand(batch_size, -1)
        item_emb = self.item_embedding(item_sequences) + self.position_embedding(positions)
        
        # Transformer encoding
        if attention_mask is not None:
            # Konverze na attention mask pro transformer (True = ignorovat)
            src_key_padding_mask = ~attention_mask
        else:
            src_key_padding_mask = None
            
        encoded = self.transformer(item_emb, src_key_padding_mask=src_key_padding_mask)
        
        # Agregace sekvence (mean pooling přes platné pozice)
        if attention_mask is not None:
            mask_expanded = attention_mask.unsqueeze(-1).float()
            seq_repr = (encoded * mask_expanded).sum(dim=1) / mask_expanded.sum(dim=1).clamp(min=1)
        else:
            seq_repr = encoded.mean(dim=1)
        
        # Kombinace s user embeddingem
        user_emb = self.user_embedding(user_ids)
        combined = torch.cat([seq_repr, user_emb], dim=-1)
        
        # Predikce skóre pro všechny položky
        scores = self.output_layer(combined)
        
        return scores


print("Model vytvořen")

In [ ]:
from torch.utils.data import Dataset, DataLoader


class UserInteractionDataset(Dataset):
    """
    Dataset pro trénování collaborative filtering modelu.
    Vytváří sekvence interakcí pro každého uživatele.
    """
    
    def __init__(
        self,
        interactions_df: pd.DataFrame,
        max_seq_length: int = 50,
        n_items: int = None
    ):
        self.max_seq_length = max_seq_length
        self.n_items = n_items or interactions_df['product_id'].max() + 1
        
        # Seřazení interakcí podle času
        sorted_df = interactions_df.sort_values(['user_id', 'timestamp'])
        
        # Seskupení podle uživatele
        self.user_sequences = {}
        self.user_targets = {}
        
        for user_id, group in sorted_df.groupby('user_id'):
            items = group['product_id'].values
            
            # Vytvoř sekvenci (input) a cílovou položku (target)
            if len(items) > 1:
                self.user_sequences[user_id] = items[:-1]  # Všechny kromě poslední
                self.user_targets[user_id] = items[-1]  # Poslední jako target
        
        self.user_ids = list(self.user_sequences.keys())
        
    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, idx):
        user_id = self.user_ids[idx]
        sequence = self.user_sequences[user_id]
        target = self.user_targets[user_id]
        
        # Padding nebo truncation
        if len(sequence) > self.max_seq_length:
            sequence = sequence[-self.max_seq_length:]  # Vezmi nejnovější
        
        # Vytvoř attention mask
        attention_mask = np.ones(len(sequence), dtype=np.float32)
        
        # Padding zleva
        pad_length = self.max_seq_length - len(sequence)
        if pad_length > 0:
            sequence = np.concatenate([np.zeros(pad_length, dtype=np.int64), sequence])
            attention_mask = np.concatenate([np.zeros(pad_length, dtype=np.float32), attention_mask])
        
        # Posunutí item IDs o 1 (0 = padding)
        sequence = sequence + 1
        
        return {
            'user_id': torch.tensor(user_id, dtype=torch.long),
            'item_sequence': torch.tensor(sequence, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.bool),
            'target': torch.tensor(target, dtype=torch.long)
        }


# Vytvoření datasetu
dataset = UserInteractionDataset(
    interactions_df,
    max_seq_length=30,
    n_items=len(products_df)
)

print(f"Dataset vytvořen: {len(dataset)} uživatelů se sekvencemi")

# Train/Val split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
# Trénování collaborative filtering modelu

n_users = len(users_df)
n_items = len(products_df)

cf_model = TransformerCollaborativeFilter(
    n_users=n_users,
    n_items=n_items,
    embedding_dim=64,
    n_heads=4,
    n_layers=2,
    max_seq_length=30
).to(device)

optimizer = torch.optim.AdamW(cf_model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
n_epochs = 20
best_val_loss = float('inf')

for epoch in range(n_epochs):
    # Training
    cf_model.train()
    train_loss = 0
    
    for batch in train_loader:
        user_ids = batch['user_id'].to(device)
        item_sequences = batch['item_sequence'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['target'].to(device)
        
        optimizer.zero_grad()
        scores = cf_model(user_ids, item_sequences, attention_mask)
        loss = criterion(scores, targets)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(cf_model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    cf_model.eval()
    val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in val_loader:
            user_ids = batch['user_id'].to(device)
            item_sequences = batch['item_sequence'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['target'].to(device)
            
            scores = cf_model(user_ids, item_sequences, attention_mask)
            loss = criterion(scores, targets)
            val_loss += loss.item()
            
            # Hit@10
            _, top_k = scores.topk(10, dim=1)
            correct += (top_k == targets.unsqueeze(1)).any(dim=1).sum().item()
            total += targets.size(0)
    
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    hit_rate = correct / total * 100
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Hit@10: {hit_rate:.1f}%")

print(f"\nTrénování dokončeno!")

## 4. Hybridní doporučovací systém

In [ ]:
@dataclass
class Recommendation:
    """Reprezentace jednoho doporučení."""
    product_id: int
    name: str
    category: str
    score: float
    source: str  # 'content', 'collaborative', 'hybrid'


class HybridRecommender:
    """
    Hybridní doporučovací systém kombinující content-based a collaborative filtering.
    """
    
    def __init__(
        self,
        content_recommender: ContentBasedRecommender,
        cf_model: TransformerCollaborativeFilter,
        products_df: pd.DataFrame,
        interactions_df: pd.DataFrame,
        device: torch.device,
        content_weight: float = 0.4,
        cf_weight: float = 0.6
    ):
        self.content_rec = content_recommender
        self.cf_model = cf_model
        self.products_df = products_df
        self.interactions_df = interactions_df
        self.device = device
        self.content_weight = content_weight
        self.cf_weight = cf_weight
        
        # Předpočítání user sekvencí
        self._build_user_sequences()
        
    def _build_user_sequences(self):
        """Vytvoří sekvence interakcí pro každého uživatele."""
        self.user_sequences = {}
        sorted_df = self.interactions_df.sort_values(['user_id', 'timestamp'])
        
        for user_id, group in sorted_df.groupby('user_id'):
            self.user_sequences[user_id] = group['product_id'].values[-30:]  # Max 30 položek
    
    def _get_content_recommendations(
        self,
        user_id: int,
        n_recommendations: int
    ) -> Dict[int, float]:
        """Získá content-based doporučení na základě historie uživatele."""
        if user_id not in self.user_sequences:
            return {}
        
        # Získej podobné produkty k těm, které uživatel viděl
        seen_products = set(self.user_sequences[user_id])
        scores = defaultdict(float)
        
        for product_id in list(seen_products)[-5]:  # Posledních 5 produktů
            similar = self.content_rec.get_similar_products(product_id, n_recommendations=10)
            for pid, score in similar:
                if pid not in seen_products:
                    scores[pid] += score
        
        # Normalizace
        if scores:
            max_score = max(scores.values())
            scores = {k: v / max_score for k, v in scores.items()}
        
        return dict(sorted(scores.items(), key=lambda x: -x[1])[:n_recommendations])
    
    @torch.no_grad()
    def _get_cf_recommendations(
        self,
        user_id: int,
        n_recommendations: int
    ) -> Dict[int, float]:
        """Získá collaborative filtering doporučení."""
        if user_id not in self.user_sequences:
            return {}
        
        self.cf_model.eval()
        
        # Příprava dat
        sequence = self.user_sequences[user_id]
        seq_length = 30
        
        # Padding
        if len(sequence) > seq_length:
            sequence = sequence[-seq_length:]
        
        attention_mask = np.ones(len(sequence), dtype=np.float32)
        pad_length = seq_length - len(sequence)
        
        if pad_length > 0:
            sequence = np.concatenate([np.zeros(pad_length, dtype=np.int64), sequence])
            attention_mask = np.concatenate([np.zeros(pad_length, dtype=np.float32), attention_mask])
        
        # Posunutí IDs
        sequence = sequence + 1
        
        # Predikce
        user_tensor = torch.tensor([user_id], dtype=torch.long).to(self.device)
        seq_tensor = torch.tensor([sequence], dtype=torch.long).to(self.device)
        mask_tensor = torch.tensor([attention_mask], dtype=torch.bool).to(self.device)
        
        scores = self.cf_model(user_tensor, seq_tensor, mask_tensor)
        scores = torch.softmax(scores, dim=1)[0].cpu().numpy()
        
        # Odstranění již viděných produktů
        seen = set(self.user_sequences[user_id])
        for pid in seen:
            if pid < len(scores):
                scores[pid] = -1
        
        # Top-K
        top_indices = np.argsort(scores)[::-1][:n_recommendations]
        
        return {int(idx): float(scores[idx]) for idx in top_indices if scores[idx] > 0}
    
    def recommend(
        self,
        user_id: int,
        n_recommendations: int = 10,
        strategy: str = 'hybrid'
    ) -> List[Recommendation]:
        """
        Vytvoří doporučení pro uživatele.
        
        Args:
            user_id: ID uživatele
            n_recommendations: Počet doporučení
            strategy: 'content', 'collaborative', nebo 'hybrid'
        """
        if strategy == 'content':
            scores = self._get_content_recommendations(user_id, n_recommendations * 2)
            source = 'content'
        elif strategy == 'collaborative':
            scores = self._get_cf_recommendations(user_id, n_recommendations * 2)
            source = 'collaborative'
        else:  # hybrid
            content_scores = self._get_content_recommendations(user_id, n_recommendations * 2)
            cf_scores = self._get_cf_recommendations(user_id, n_recommendations * 2)
            
            # Kombinace skóre
            all_products = set(content_scores.keys()) | set(cf_scores.keys())
            scores = {}
            
            for pid in all_products:
                content_s = content_scores.get(pid, 0)
                cf_s = cf_scores.get(pid, 0)
                scores[pid] = self.content_weight * content_s + self.cf_weight * cf_s
            
            source = 'hybrid'
        
        # Vytvoření výsledků
        results = []
        sorted_scores = sorted(scores.items(), key=lambda x: -x[1])[:n_recommendations]
        
        for pid, score in sorted_scores:
            product = self.products_df[self.products_df['product_id'] == pid]
            if len(product) > 0:
                product = product.iloc[0]
                results.append(Recommendation(
                    product_id=pid,
                    name=product['name'],
                    category=product['category'],
                    score=score,
                    source=source
                ))
        
        return results


# Vytvoření hybridního doporučovače
hybrid_recommender = HybridRecommender(
    content_recommender=content_recommender,
    cf_model=cf_model,
    products_df=products_df,
    interactions_df=interactions_df,
    device=device,
    content_weight=0.4,
    cf_weight=0.6
)

# Test doporučení
test_user_id = 0
print(f"Doporučení pro uživatele {test_user_id}:")
print(f"\nHistorie uživatele:")
user_history = hybrid_recommender.user_sequences.get(test_user_id, [])
for pid in user_history[-5:]:
    product = products_df[products_df['product_id'] == pid].iloc[0]
    print(f"  - {product['name']} ({product['category']})")

print(f"\nHybridní doporučení:")
recommendations = hybrid_recommender.recommend(test_user_id, n_recommendations=5, strategy='hybrid')
for rec in recommendations:
    print(f"  {rec.name} ({rec.category}) - skóre: {rec.score:.3f}")

## 5. Produkční implementace

In [ ]:
from datetime import datetime
import hashlib
import json


class ProductionRecommenderSystem:
    """
    Produkční doporučovací systém s cachingem, logováním a A/B testováním.
    """
    
    def __init__(
        self,
        hybrid_recommender: HybridRecommender,
        cache_size: int = 1000,
        cache_ttl_minutes: int = 30
    ):
        self.recommender = hybrid_recommender
        self.cache = {}
        self.cache_size = cache_size
        self.cache_ttl = cache_ttl_minutes * 60
        
        # Statistiky
        self.stats = {
            'total_requests': 0,
            'cache_hits': 0,
            'cache_misses': 0,
            'strategy_usage': defaultdict(int),
            'avg_latency_ms': 0,
            'latencies': []
        }
        
        # A/B test konfigurace
        self.ab_tests = {}
        
    def _get_cache_key(self, user_id: int, strategy: str, n: int) -> str:
        """Generuje klíč pro cache."""
        return hashlib.md5(f"{user_id}:{strategy}:{n}".encode()).hexdigest()
    
    def _is_cache_valid(self, cache_entry: dict) -> bool:
        """Kontroluje platnost cache záznamu."""
        if not cache_entry:
            return False
        age = (datetime.now() - cache_entry['timestamp']).total_seconds()
        return age < self.cache_ttl
    
    def get_recommendations(
        self,
        user_id: int,
        n_recommendations: int = 10,
        strategy: Optional[str] = None,
        context: Optional[Dict] = None
    ) -> Dict:
        """
        Získá doporučení s produkčními features.
        
        Args:
            user_id: ID uživatele
            n_recommendations: Počet doporučení
            strategy: Strategie ('content', 'collaborative', 'hybrid') nebo None pro A/B test
            context: Dodatečný kontext (stránka, čas, zařízení)
        """
        start_time = datetime.now()
        self.stats['total_requests'] += 1
        
        # A/B test - pokud není zadána strategie
        if strategy is None:
            strategy = self._select_ab_variant(user_id)
        
        self.stats['strategy_usage'][strategy] += 1
        
        # Kontrola cache
        cache_key = self._get_cache_key(user_id, strategy, n_recommendations)
        if cache_key in self.cache and self._is_cache_valid(self.cache[cache_key]):
            self.stats['cache_hits'] += 1
            return self.cache[cache_key]['data']
        
        self.stats['cache_misses'] += 1
        
        # Získání doporučení
        recommendations = self.recommender.recommend(
            user_id=user_id,
            n_recommendations=n_recommendations,
            strategy=strategy
        )
        
        # Příprava výsledku
        result = {
            'user_id': user_id,
            'recommendations': [
                {
                    'product_id': rec.product_id,
                    'name': rec.name,
                    'category': rec.category,
                    'score': round(rec.score, 4),
                    'rank': i + 1
                }
                for i, rec in enumerate(recommendations)
            ],
            'strategy': strategy,
            'timestamp': datetime.now().isoformat(),
            'context': context or {}
        }
        
        # Uložení do cache
        self._update_cache(cache_key, result)
        
        # Aktualizace latence
        latency_ms = (datetime.now() - start_time).total_seconds() * 1000
        self._update_latency(latency_ms)
        
        return result
    
    def _update_cache(self, key: str, data: dict):
        """Aktualizuje cache s LRU eviction."""
        if len(self.cache) >= self.cache_size:
            # Smaž nejstarší záznam
            oldest_key = min(self.cache.keys(), key=lambda k: self.cache[k]['timestamp'])
            del self.cache[oldest_key]
        
        self.cache[key] = {
            'data': data,
            'timestamp': datetime.now()
        }
    
    def _update_latency(self, latency_ms: float):
        """Aktualizuje průměrnou latenci."""
        self.stats['latencies'].append(latency_ms)
        # Udržuj jen posledních 1000 měření
        if len(self.stats['latencies']) > 1000:
            self.stats['latencies'] = self.stats['latencies'][-1000:]
        self.stats['avg_latency_ms'] = sum(self.stats['latencies']) / len(self.stats['latencies'])
    
    def _select_ab_variant(self, user_id: int) -> str:
        """Vybere A/B test variantu na základě user_id."""
        # Deterministické přiřazení na základě hash user_id
        hash_val = hash(user_id) % 100
        
        if hash_val < 33:
            return 'content'
        elif hash_val < 66:
            return 'collaborative'
        else:
            return 'hybrid'
    
    def log_interaction(
        self,
        user_id: int,
        product_id: int,
        interaction_type: str,
        recommendation_rank: Optional[int] = None
    ):
        """
        Zaloguje interakci uživatele (pro vyhodnocení kvality doporučení).
        """
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'user_id': user_id,
            'product_id': product_id,
            'interaction_type': interaction_type,
            'recommendation_rank': recommendation_rank,
            'was_recommended': recommendation_rank is not None
        }
        # V produkci by se logoval do databáze/Kafka
        return log_entry
    
    def get_statistics(self) -> Dict:
        """Vrátí statistiky systému."""
        return {
            'total_requests': self.stats['total_requests'],
            'cache_hit_rate': self.stats['cache_hits'] / max(1, self.stats['total_requests']) * 100,
            'avg_latency_ms': round(self.stats['avg_latency_ms'], 2),
            'strategy_distribution': dict(self.stats['strategy_usage']),
            'cache_size': len(self.cache)
        }


# Vytvoření produkčního systému
production_system = ProductionRecommenderSystem(
    hybrid_recommender=hybrid_recommender,
    cache_size=100,
    cache_ttl_minutes=15
)

# Simulace provozu
print("Simulace produkčního provozu:")
print("=" * 50)

for user_id in range(10):
    result = production_system.get_recommendations(
        user_id=user_id,
        n_recommendations=5,
        context={'page': 'homepage', 'device': 'mobile'}
    )
    
    print(f"\nUživatel {user_id} (strategie: {result['strategy']}):")
    for rec in result['recommendations'][:3]:
        print(f"  {rec['rank']}. {rec['name']} (skóre: {rec['score']})")

# Druhý průchod pro test cache
print("\n" + "=" * 50)
print("Opakované požadavky (test cache):")
for user_id in range(5):
    production_system.get_recommendations(user_id, 5)

print(f"\nStatistiky systému:")
for key, value in production_system.get_statistics().items():
    print(f"  {key}: {value}")

In [ ]:
# API-like interface pro integraci

class RecommenderAPI:
    """
    API wrapper pro snadnou integraci do webových aplikací.
    """
    
    def __init__(self, production_system: ProductionRecommenderSystem):
        self.system = production_system
        
    def get_homepage_recommendations(self, user_id: int) -> Dict:
        """Doporučení pro homepage."""
        return self.system.get_recommendations(
            user_id=user_id,
            n_recommendations=10,
            context={'page': 'homepage'}
        )
    
    def get_similar_products(self, product_id: int, n: int = 5) -> List[Dict]:
        """Podobné produkty (pro product detail page)."""
        similar = self.system.recommender.recommender.content_rec.get_similar_products(
            product_id, n
        )
        
        products_df = self.system.recommender.products_df
        results = []
        
        for pid, score in similar:
            product = products_df[products_df['product_id'] == pid]
            if len(product) > 0:
                product = product.iloc[0]
                results.append({
                    'product_id': pid,
                    'name': product['name'],
                    'category': product['category'],
                    'score': round(score, 4)
                })
        
        return results
    
    def search(self, query: str, n: int = 10) -> List[Dict]:
        """Vyhledávání produktů."""
        results = self.system.recommender.recommender.content_rec.search_products(
            query, n
        )
        
        products_df = self.system.recommender.products_df
        formatted = []
        
        for pid, score in results:
            product = products_df[products_df['product_id'] == pid]
            if len(product) > 0:
                product = product.iloc[0]
                formatted.append({
                    'product_id': pid,
                    'name': product['name'],
                    'category': product['category'],
                    'price': int(product['price']),
                    'relevance_score': round(score, 4)
                })
        
        return formatted
    
    def track_click(self, user_id: int, product_id: int, rank: int):
        """Zaloguje klik na doporučení."""
        return self.system.log_interaction(
            user_id=user_id,
            product_id=product_id,
            interaction_type='click',
            recommendation_rank=rank
        )


# Demo API
api = RecommenderAPI(production_system)

print("=== Demo API ===")
print("\n1. Homepage doporučení:")
homepage_recs = api.get_homepage_recommendations(user_id=1)
for rec in homepage_recs['recommendations'][:3]:
    print(f"   - {rec['name']}")

print("\n2. Podobné produkty (k iPhone):")
similar = api.get_similar_products(product_id=1, n=3)
for item in similar:
    print(f"   - {item['name']} ({item['category']})")

print("\n3. Vyhledávání 'fitness':")
search_results = api.search("fitness vybavení", n=3)
for item in search_results:
    print(f"   - {item['name']} - {item['price']} Kč")

## Shrnutí

V tomto notebooku jsme vytvořili kompletní doporučovací systém:

1. **Content-based doporučování** - Využívá Sentence Transformers pro embedding produktových popisů
2. **Collaborative filtering** - Transformer model pro modelování sekvencí uživatelských interakcí
3. **Hybridní přístup** - Kombinace obou metod s váhovanými skóre
4. **Produkční features** - Caching, A/B testování, logování interakcí

### Klíčové poznatky

- Content-based je vhodný pro cold-start (nové produkty, noví uživatelé)
- Collaborative filtering lépe zachycuje latentní preference uživatelů
- Hybridní přístup poskytuje nejrobustnější výsledky
- Cache je kritická pro produkční výkon (snížení latence o 10-100x)

### Praktické tipy

- Vždy implementujte fallback strategie (např. populární produkty)
- A/B testování je nutné pro měření reálného dopadu
- Logujte interakce pro kontinuální zlepšování modelu
- Pravidelně přetrénujte model na nových datech